In [1]:
from dnn.data.dataloader import init_default_loader
from dnn.data.dataset import SpeechDataset
from dnn.utils import secToSample, secToFrames
from dnn.sv_score import embeds

import torch.nn.functional as F
import os
import pandas as pd
import numpy as np

In [2]:
from dnn.utils.parser import get_sv_parser
options = get_sv_parser().parse_args(args=[])
options.n_dct_filters = 40
options.n_mels = 40
options.timeshift_ms = 100
options.window_size= 0.025
options.window_stride= 0.010
options.cache_size = 32768

In [21]:
options.input_format = 'fbank'
options.input_clip = True
options.input_length = secToSample(4)
options.splice_frames = secToSample(0.1)//160+1

In [4]:
options.data_folder = "/home/muncok/DL/projects/human_sv/wavs/reddots/"

In [23]:
from dnn.model.SpeechModel import SpeechResModel
model = SpeechResModel("res15", 10)
model.load_partial("../interspeech2018/models/voxc/si_train/full_train/si_voxc_res15_0.1s_full_fbank.pt")
model.cuda()
lda = None  # None means not using lda

loaded from ../interspeech2018/models/voxc/si_train/full_train/si_voxc_res15_0.1s_full_fbank.pt


In [24]:
test_samples = []
for dirpath, dirnames, filenames in os.walk("../human_sv/wavs/reddots/"):
    filenames = [file for file in filenames if file.endswith(".wav")]
    for file in filenames:
        test_samples.append(os.path.abspath(os.path.join(dirpath,file)))
test_samples = sorted(test_samples)        

samples = [sample.split("/")[-2:] + [os.path.join(*sample.split("/")[-2:])] for sample in test_samples]
test_df = pd.DataFrame.from_records(samples, columns=['spk', 'file', 'path'])
usage = test_df.file.apply(lambda x: "enroll" if x.startswith("e") else "test")
test_df['usage'] = usage
data = SpeechDataset.read_df(options.data_folder, test_df)

In [25]:
config = vars(options)
dataset = SpeechDataset(data, "test", config)
val_dataloader = init_default_loader(dataset)

In [26]:
embeddings, _ = embeds(options, val_dataloader, model, lda)

In [27]:
import torch
spks = test_df.spk.unique().tolist()
for spk in spks:
    print(spk)
    enroll_idxs = np.nonzero((test_df.spk == spk) & (test_df.usage == "enroll"))
    test_idxs = np.nonzero((test_df.spk == spk) & (test_df.usage == "test"))
    enroll_embeddings = torch.mean(embeddings[enroll_idxs], 0, True)
    test_embeddings = embeddings[test_idxs]
    print(F.cosine_similarity(enroll_embeddings, test_embeddings))

A_m0001_31
tensor([ 0.9878,  0.9853,  0.9463,  0.9869,  0.9898,  0.9264,  0.9394,
         0.9919,  0.9790,  0.9909])
B_m0015_40
tensor([ 0.9944,  0.9733,  0.9694,  0.9941,  0.9054,  0.9911,  0.9634,
         0.9449,  0.9610,  0.9899])
C_m0008_39
tensor([ 0.9946,  0.9831,  0.9873,  0.9260,  0.9314,  0.9921,  0.9237,
         0.9154,  0.9794,  0.9912])


In [34]:
torch.cuda.set_device(0)

In [30]:
t = iter(val_dataloader).__next__()

In [32]:
t[0].shape

torch.Size([39, 401, 40])